In [1]:
import spacy
from spacy import displacy
import numpy as np
import pandas as pd

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
# from google.colab import files


# uploaded = files.upload()


In [4]:
df = pd.read_csv('heuristics.csv',header=None)
print(df)


                                                    0
0   Where immediately before the issue of a notifi...
1   Where in respect of an industrial dispute rela...
2   At the time of hearing, the authority may dire...
3   The authority referred to in sub-section (6) m...
4   Finality of orders constituting Board, Committ...
5   Chief Inspector- The State Government may, by ...
6   Where the applicant succeeds in such proceedin...
7   Penalty for obstructing Inspector. -Whoever wi...
8   Protection to persons acting under the Act. - ...
9   Transfer of cases from one Court to another. -...
10  Removal of difficulties.-If any difficulty ari...
11  Provided that the appropriate Government may, ...
12  Provided that in relation to the State of Jamm...
13  Provided further that when the provisions of t...
14  Every employee shall be entitled to be paid by...
15  Subject to the other provisions of this Act, e...
16  Where in respect of any accounting year referr...
17  In computing the allocab

In [5]:
df.head()

,0
0,Where immediately before the issue of a notifi...
1,Where in respect of an industrial dispute rela...
2,"At the time of hearing, the authority may dire..."
3,The authority referred to in sub-section (6) m...
4,"Finality of orders constituting Board, Committ..."


In [6]:
i = 3
for index,row in df.iterrows():
    sentence = row.values[0]
    doc = nlp(sentence)
    print(i,"\t")
    for token in doc:
        ancestors = [t.text for t in token.ancestors]
        children = [t.text for t in token.children]
        if(token.dep_ == "ROOT"):
            print(token.text, "\t", 
          token.pos_, "\t" )
    i = i + 1
    print("\n")

3 	
be 	 AUX 	


4 	
apply 	 VERB 	


5 	
direct 	 VERB 	
paid 	 VERB 	


6 	
allow 	 VERB 	


7 	
Finality 	 NOUN 	
called 	 VERB 	


8 	
appoint 	 VERB 	


9 	
calculate 	 VERB 	
be 	 AUX 	


10 	
Penalty 	 NOUN 	
obstructs 	 VERB 	


11 	
Protection 	 NOUN 	
lie 	 VERB 	


12 	
Transfer 	 NOUN 	
- 	 PUNCT 	
subordinate 	 ADJ 	


13 	
do 	 VERB 	


14 	
Provided 	 VERB 	
employing 	 VERB 	


15 	
Provided 	 VERB 	


16 	
Provided 	 VERB 	


17 	
entitled 	 VERB 	


18 	
bound 	 VERB 	


19 	
bound 	 VERB 	


20 	
taken 	 VERB 	


21 	
calculated 	 VERB 	


22 	
reduced 	 VERB 	


23 	
exceeds 	 VERB 	
carried 	 VERB 	


24 	
carried 	 VERB 	


25 	
apply 	 VERB 	


26 	
taken 	 VERB 	


27 	
Where 	 SCONJ 	


28 	
be 	 AUX 	


29 	
apply 	 VERB 	


30 	
be 	 AUX 	


31 	
sells 	 VERB 	


32 	
make 	 VERB 	


33 	
Provided 	 VERB 	


34 	
Provided 	 VERB 	


35 	
deemed 	 VERB 	


36 	
14 	 NUM 	
[ 	 X 	
referred 	 VERB 	


37 	
comply 	 VERB 	


38 	
specified 	 VERB 	


39 	
enable 

In [7]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    children = [t.text for t in token.children]
    if(token.dep_ == "ROOT"):
        print(token.text, "\t", token.i, "\t", 
          token.pos_, "\t", token.dep_, "\t", 
          ancestors, "\t", children)

take 	 3 	 VERB 	 ROOT 	 [] 	 ['court', 'shall', 'cognizance', ',', 'save']
authorised 	 76 	 VERB 	 ROOT 	 [] 	 ['[', 'specially', 'in', 'by']


In [8]:
sentence = "Protection to persons acting under the Act. - No suit, prosecution or other legal proceeding whatsoever shall lie against any person for anything which is in good faith done or intended to be done in due discharge of his duties under this Act."

In [9]:
doc = nlp(sentence)

In [10]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    children = [t.text for t in token.children]
    if(token.dep_ == "ROOT"):
        print(token.text, "\t", token.i, "\t", 
            token.pos_, "\t", token.dep_, "\t", 
            ancestors, "\t", children)

Protection 	 0 	 NOUN 	 ROOT 	 [] 	 ['to', '.', '-']
lie 	 19 	 VERB 	 ROOT 	 [] 	 ['suit', 'shall', 'against', 'for', '.']


In [11]:
def find_root_of_sentence(doc):
    root_token = None
    for token in doc:
        if (token.dep_ == "ROOT"):
            root_token = token
    return root_token

In [12]:
root_token = find_root_of_sentence(doc)

In [13]:
def find_other_verbs(doc, root_token):
    other_verbs = []
    for token in doc:
        ancestors = list(token.ancestors)
        if (token.pos_ == "VERB" and len(ancestors) == 1\
            and ancestors[0] == root_token):
            other_verbs.append(token)
    return other_verbs

In [14]:
other_verbs = find_other_verbs(doc, root_token)

In [15]:
def get_clause_token_span_for_verb(verb, doc, all_verbs):
    first_token_index = len(doc)
    last_token_index = 0
    this_verb_children = list(verb.children)
    for child in this_verb_children:
        if (child not in all_verbs):
            if (child.i < first_token_index):
                first_token_index = child.i
            if (child.i > last_token_index):
                last_token_index = child.i
    return(first_token_index, last_token_index)

In [16]:
token_spans = []   
all_verbs = [root_token] + other_verbs
for other_verb in all_verbs:
    (first_token_index, last_token_index) = \
     get_clause_token_span_for_verb(other_verb, 
                                    doc, all_verbs)
    token_spans.append((first_token_index, 
                        last_token_index))

In [17]:
sentence_clauses = []
for token_span in token_spans:
    start = token_span[0]
    end = token_span[1]
    if (start < end):
        clause = doc[start:end]
        sentence_clauses.append(clause)
sentence_clauses = sorted(sentence_clauses, 
                          key=lambda tup: tup[0])

In [18]:
clauses_text = [clause.text for clause in sentence_clauses]
print(clauses_text)

['suit, prosecution or other legal proceeding whatsoever shall lie against any person for anything which is in good faith done or intended to be done in due discharge of his duties under this Act']


# **REPLICATING ONLINE SPACY**

DHRUV'S AIM-ATTRIBUTE-DEONTIC

In [19]:
nlp = spacy.load("en_core_web_sm")
sentence = "Provided that nothing contained in this sub-section shall render any such person liable to any punishment if he proves that the offence was committed without his knowledge or that he exercised all; due diligence to prevent the commission of such offence"
doc = nlp(sentence)


def merge_phrases(doc):
    with doc.retokenize() as retokenizer:
        for np in list(doc.noun_chunks):
            attrs = {
                "tag": np.root.tag_,
                "lemma": np.root.lemma_,
                "ent_type": np.root.ent_type_,
            }
            retokenizer.merge(np, attrs=attrs)
    return doc

def merge_punct(doc):
    spans = []
    for word in doc[:-1]:
        if word.is_punct or not word.nbor(1).is_punct:
            continue
        start = word.i
        end = word.i + 1
        while end < len(doc) and doc[end].is_punct:
            end += 1
        span = doc[start:end]
        spans.append((span, word.tag_, word.lemma_, word.ent_type_))
    with doc.retokenize() as retokenizer:
        for span, tag, lemma, ent_type in spans:
            attrs = {"tag": tag, "lemma": lemma, "ent_type": ent_type}
            retokenizer.merge(span, attrs=attrs)
    return doc

# Merge noun phrases into one token.
doc = merge_phrases(doc)
# Attach punctuation to tokens
doc = merge_punct(doc)

In [20]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    ancestors_token = [t for t in token.ancestors]
    children = [t.text for t in token.children]
    children_token = [t for t in token.children]
    left = [t.text for t in token.lefts]
    right = [t.text for t in token.rights]
    print(token.text, token.pos_, token.dep_,ancestors,children,left,right)


Provided VERB prep ['diligence'] ['render'] [] ['render']
that SCONJ mark ['render', 'Provided', 'diligence'] [] [] []
nothing PRON nsubj ['render', 'Provided', 'diligence'] ['contained'] [] ['contained']
contained VERB acl ['nothing', 'render', 'Provided', 'diligence'] ['in'] [] ['in']
in ADP prep ['contained', 'nothing', 'render', 'Provided', 'diligence'] ['this sub-'] [] ['this sub-']
this sub- NOUN pobj ['in', 'contained', 'nothing', 'render', 'Provided', 'diligence'] [] [] []
section NOUN nsubj ['render', 'Provided', 'diligence'] [] [] []
shall AUX aux ['render', 'Provided', 'diligence'] [] [] []
render VERB ccomp ['Provided', 'diligence'] ['that', 'nothing', 'section', 'shall', 'liable', 'proves'] ['that', 'nothing', 'section', 'shall'] ['liable', 'proves']
any such person NOUN nsubj ['liable', 'render', 'Provided', 'diligence'] [] [] []
liable ADJ ccomp ['render', 'Provided', 'diligence'] ['any such person', 'to'] ['any such person'] ['to']
to ADP prep ['liable', 'render', 'Prov

In [21]:
ners = [(e,type(e)) for e in doc.ents]
ners

[]

In [22]:
nlp.pipeline
# Tagger identifies verb, noun etc
# Parser identifies relationships b/w different tokens
# ner labels named entities

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1ef4ab5bd00>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1ef4ab5ae00>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1ef4ab910e0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1ef4be40640>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1ef4be05680>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1ef4ab90eb0>)]

In [23]:
attribute = []
attribute_token = []
aim = []
aim_token = []
deontic = []
deontic_token = []
aim_2 = []
aim_2_token = []


For Attribute

In [24]:
i = 0
for entity in doc:
    print(entity)
    if ((entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or "PRON") and (entity.dep_ == "nsubjpass" or entity.dep_ == "nsubj")):
        attribute.append(entity.text)
        attribute_token.append(entity)
        for attr in entity.children:
            if attr.dep_ == 'conj' and (attr.pos_ == "PROPN" or attr.pos_ == "NOUN"):
                attribute.append(attr.text)
                attribute_token.append(attr)
    
    if(len(attribute)>0):
        break

Provided
that
nothing


In [25]:
print(attribute_token)

[nothing]


For Deontic

In [26]:
for entity in doc:
    print(entity)
    if ((entity.pos_ == "AUX") and (entity.dep_ == "aux" or entity.dep_ == "auxpass")):
        deontic.append(entity.text)
        deontic_token.append(entity)
        break
  
  

Provided
that
nothing
contained
in
this sub-
section
shall


In [27]:
print(deontic)

['shall']


For Aim

In [28]:
for entity in doc:
  if (entity.pos_ =='NOUN' or entity.pos_ =='VERB'):
    for child in entity.children:
      if (child in attribute_token) and (child.dep_ == 'nsubj' or child.dep_ =='nsubjpass'):
        aim.append(entity.text)
        aim_token.append(entity)

        for attr in entity.children:
          if attr.dep_ == 'conj' and (attr.pos_ == "NOUN" or attr.pos_ == "VERB"):
            aim.append(attr.text)
            aim_token.append(attr)
  
  if(len(aim)>0):
    break

In [29]:
print(aim)

['render']


Aim Updated code

In [30]:
for entity in doc:
  if (entity.pos_ =='VERB'):
    for child in entity.children:
      if (child in deontic_token) and (child.dep_ == 'aux'):
        aim_2.append(entity.text)
        aim_2_token.append(entity)

        for attr in entity.children:
          if attr.dep_ == 'conj' and (attr.pos_ == "NOUN" or attr.pos_ == "VERB"):
            aim.append(attr.text)
            aim_token.append(attr)
  
  if(len(aim_2)>0):
    break

In [31]:
print(aim_2)

['render']


## Heuristic 2


In [32]:
sentence = "Where in respect of an industrial dispute relating to the rates of wages payable to any of the employees employed in a scheduled employment, any proceeding is pending before a Tribunal or National Tribunal under the Industrial Disputes Act, 1947 (24 of 1947) or before any like authority under any other law for the time being in force, or an award made by any Tribunal, National Tribunal or such authority is in operation, and a notification fixing or revising the minimum rates of wages in respect of the scheduled employment is issued during the pendency of such proceeding or the operation of the award, then, notwithstanding anything contained in this Act, the minimum rates of wages so fixed or so revised shall not apply to those employees during the period in which the proceeding is pending and the award made therein is in operation or, as the case may be, where the notification is issued during the period of operation of an award, during that period; and where such proceeding or award relates to the rates of wages payable to all the employees in the scheduled employment, no minimum rates of wages shall be fixed or revised in respect of that employment during the said period."
doc = nlp(sentence)

In [33]:
# Merge noun phrases into one token.
doc = merge_phrases(doc)
# Attach punctuation to tokens
doc = merge_punct(doc)

In [34]:
root_tokens = []

In [35]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    ancestors_token = [t for t in token.ancestors]
    children = [t.text for t in token.children]
    children_token = [t for t in token.children]
    left = [t.text for t in token.lefts]
    right = [t.text for t in token.rights]
    if(token.dep_ == "ROOT"):
        root_tokens.append(token)
        print(token.text, "\t", 
          token.pos_, "\t" )
        print("\n")
    print(token.text, token.pos_, token.dep_,ancestors,children)


Where SCONJ advmod ['in', 'is', 'apply'] []
in ADP prep ['is', 'apply'] ['Where', 'respect']
respect NOUN pobj ['in', 'is', 'apply'] ['of']
of ADP prep ['respect', 'in', 'is', 'apply'] ['an industrial dispute']
an industrial dispute NOUN pobj ['of', 'respect', 'in', 'is', 'apply'] ['relating']
relating VERB acl ['an industrial dispute', 'of', 'respect', 'in', 'is', 'apply'] ['to']
to ADP prep ['relating', 'an industrial dispute', 'of', 'respect', 'in', 'is', 'apply'] ['the rates']
the rates NOUN pobj ['to', 'relating', 'an industrial dispute', 'of', 'respect', 'in', 'is', 'apply'] ['of']
of ADP prep ['the rates', 'to', 'relating', 'an industrial dispute', 'of', 'respect', 'in', 'is', 'apply'] ['wages']
wages NOUN pobj ['of', 'the rates', 'to', 'relating', 'an industrial dispute', 'of', 'respect', 'in', 'is', 'apply'] ['payable']
payable ADJ amod ['wages', 'of', 'the rates', 'to', 'relating', 'an industrial dispute', 'of', 'respect', 'in', 'is', 'apply'] ['to']
to ADP prep ['payable', '

In [36]:
root_tokens

[apply]

In [37]:
root_verb = []
root_aux = []
root_noun = []
root_others = []
case = 5

for token in root_tokens:
    if(token.pos_ == 'VERB'):
        root_verb.append(token)
    elif(token.pos_ == 'AUX'):
        root_aux.append(token)
    elif(token.pos_ == 'NOUN'):
        root_noun.append(token)
    else:
        root_others.append(token)

if(len(root_verb)>0):
    case = 1
elif(len(root_aux)>0):
    case = 2
elif(len(root_noun)>0):
    case = 3
elif(len(root_others)>0):
    case = 4
print(case)

1


### CASE 1


In [38]:
aim_token = []
attr_token = []
deontic_token = []

**AIM**

In [39]:
#working on root_verb
flag = 0
#first adding all VERBS with conj and ccomp to the root_verb list
for token in root_verb:
    for children in token.children:
        if(children.dep_ == 'ccomp' or children.dep_ == 'conj'):
            root_verb.append(children)

#in GIVEN ORDER in the list, if we find a token with both nsubj and aux then we return it
for token in root_verb:
    children_dep = [child.dep_ for child in token.children]
    str1 = "nsubj"
    str2 = "aux"
    
    if(str1 in children_dep and str2 in children_dep):
        print(token.text)
        aim_token.append(token)
        flag = 1
        break #only 1 aim in this case
#nusbjpass and aux if not aux and nsubj

apply


In [40]:
#if we dont find any such VERB then we take entire root_verb as aim (some of them might be right as there could be multiple)
if(flag == 0):
    aim_token = root_verb
print(aim_token)
#NOTE: AIM never empty since CASE 1
#ALSO 

[apply]


**DEONTIC**

In [41]:
keywords = ["shall","may","must"]
secondary_keywords = ["is","has","have","was","having"]
#deontic won't be empty due to keywords and secondary keywords
for token in aim_token:
    for child in token.children:
        if(child.dep_ =='aux' and child.pos_ == 'AUX'):
            deontic_token.append(child)
            break
    if(len(deontic_token)>0):
        break

#if no such deontic found then search in keywords and later in secondary keywords
if(len(deontic_token) == 0):
    for keyword in keywords:
        for token in doc:
            if(keyword in token.text and token.pos_ =="AUX"):
                deontic_token.append(token)
                break
        if(len(deontic_token)>0):
            break

if(len(deontic_token) == 0):
    for keyword in secondary_keywords:
        for token in doc:
            if(keyword in token.text and token.pos_ =="AUX"):
                deontic_token.append(token)
                break
        if(len(deontic_token)>0):
            break

In [42]:
print(deontic_token)

[shall]


In [43]:
#first check for nsubj
for token in aim_token:
    for entity in token.children:
        if(entity.dep_ =='nsubj' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
            attr_token.append(entity)

#else check for nsubpass
if(len(attr_token) == 0):
    for token in aim_token:
        for entity in token.children:
            if(entity.dep_ =='nsubjpass' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                attr_token.append(entity)

#else if empty then check in entire sentence
if(len(attr_token) == 0):
    for token in doc:
        for entity in token.children:
            if(entity.dep_ =='nsubj' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                attr_token.append(entity)
                break
        if(len(attr_token) > 0):
            break

if(len(attr_token) == 0):
    for token in doc:
        for entity in token.children:
            if(entity.dep_ =='nsubjpass' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                attr_token.append(entity)
                break
        if(len(attr_token) > 0):
            break

In [44]:
print(attr_token)

[the minimum rates]


### CASE 2

In [45]:
aim_token = []
attr_token = []
deontic_token = []

**AIM**

In [46]:
for token in root_aux:
    for children in token.children:
        if((children.dep_ == 'ccomp' or children.dep_ == 'conj') and children.pos_ == 'VERB'):
            aim_token.append(children)

if(len(aim_token) == 0):
    for children in token.children:
        if((children.dep_ == 'advcl') and children.pos_ == 'NOUN'):
            aim_token.append(children)
    for token in aim_token:
        for children in token.children:
            if((children.dep_ == 'ccomp' or children.dep_ == 'conj') and children.pos_ == 'VERB'):
                aim_token.append(children)

In [47]:
print(aim_token)

[]


**DEONTIC**

In [48]:
#for deontic first check aux from root_aux else check from aim else from keywords and secondary keywords
for token in root_aux:
    for children in token.children:
        if((children.dep_ == 'aux') and children.pos_ == 'AUX'):
            deontic_token.append(children)
            break
    if(len(deontic_token) > 0):
        break

if(len(deontic_token) == 0):
    for token in aim_token:
        for child in token.children:
            if(child.dep_ =='aux' and child.pos_ == 'AUX'):
                deontic_token.append(child)
                break
        if(len(deontic_token)>0):
            break

#if no such deontic found then search in keywords and later in secondary keywords
if(len(deontic_token) == 0):
    for keyword in keywords:
        for token in doc:
            if(keyword in token.text and token.pos_ =="AUX"):
                deontic_token.append(token)
                break
        if(len(deontic_token)>0):
            break

if(len(deontic_token) == 0):
    for keyword in secondary_keywords:
        for token in doc:
            if(keyword in token.text and token.pos_ =="AUX"):
                deontic_token.append(token)
                break
        if(len(deontic_token)>0):
            break

In [49]:
print(deontic_token)

[shall]


**ATTRIBUTE**

In [50]:
for token in root_aux:
    for entity in token.children:
        if(entity.dep_ =='nsubj' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
            attr_token.append(entity)

for token in aim_token:
    for entity in token.children:
        if(entity.dep_ =='nsubj' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
            attr_token.append(entity)

#else check for nsubpass
if(len(attr_token) == 0):
    for token in aim_token:
        for entity in token.children:
            if(entity.dep_ =='nsubjpass' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                attr_token.append(entity)

#else if empty then check in entire sentence
if(len(attr_token) == 0):
    for token in doc:
        for entity in token.children:
            if(entity.dep_ =='nsubj' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                attr_token.append(entity)
                break
        if(len(attr_token) > 0):
            break

if(len(attr_token) == 0):
    for token in doc:
        for entity in token.children:
            if(entity.dep_ =='nsubjpass' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                attr_token.append(entity)
                break
        if(len(attr_token) > 0):
            break

In [51]:
print(attr_token)

[any proceeding]


**CASE 4**

In [52]:
entity_set = []

In [53]:
#trying to extract the most common case - a verb with nsubj AND aux both OR try heuristic1
for token  in doc:
    if(token.pos_ == 'VERB'):
        child_dependency = [child.dep_ for child in token.children]
        str1 = "aux"
        str2 = "nsubj"
        if((str1 in child_dependency) and (str2 in child_dependency)):
            req_token = []
            req_token.append(token)
            for dependencies in token.children:
                if(dependencies.dep_ == 'aux'):
                    req_token.append(dependencies)
                    break
            for dependencies in token.children:
                if(dependencies.dep_ == 'nsubj'):
                    req_token.append(dependencies)
                    break
            entity_set.append(req_token)

In [54]:
print(entity_set)

[[apply, shall, the minimum rates]]
